## 一、生成逻辑

In [1]:
from illufly.community.openai import OpenAIEmbeddings
from illufly.community.chroma.chroma_db import ChromaDB
from illufly.community.openai import ChatOpenAI
from illufly.memory.kg import KnowledgeGraph
from illufly.utils import extract_segments
from illufly.rocksdb import IndexedRocksDB

import logging
logging.basicConfig(level=logging.INFO, force=True)

db_path = "__mydb"
user_id = "12345678"

docs_db = IndexedRocksDB(db_path)
vdb = ChromaDB(embeddings=OpenAIEmbeddings(db=docs_db))
openai = ChatOpenAI(imitator="QWEN", model="qwen-plus")
kg = KnowledgeGraph(llm=openai, vector_db=vdb, docs_db=docs_db)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [4]:
await kg.init(user_id)

INFO:illufly.memory.kg:[12345678:8d8c198ded61dd169bb77c4498c0882d] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:广州鸿蒙 m:主营业务 "税务局客户信息化服务" .
INFO:illufly.memory.kg:[12345678:dbfafea0a20896359a023fffeda044e0] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:广州鸿蒙 m:企业年龄 "15" .
INFO:illufly.memory.kg:[12345678:dbdf57081dd7ca344c6e557e59c0c9e5] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:广州鸿蒙 m:成立年份 "2008" .
INFO:illufly.memory.kg:[3] 条知识已加载


### 生成Turtle表达式

In [5]:
await kg.extract(
    "广州鸿蒙成立于2008年，是一家15年的信息化企业，主营业务是做税务局客户",
    user_id=user_id
)

INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
INFO:illufly.memory.kg:[12345678:8d8c198ded61dd169bb77c4498c0882d] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:广州鸿蒙 m:主营业务 "税务局客户信息化服务" .
INFO:illufly.memory.kg:[12345678:dbfafea0a20896359a023fffeda044e0] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:广州鸿蒙 m:企业年龄 "15" .
INFO:illufly.memory.kg:[12345678:dbdf57081dd7ca344c6e557e59c0c9e5] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:广州鸿蒙 m:成立年份 "2008" .


'@prefix m: <http://illufly.com/12345678/memory#> .\n@prefix prov: <http://www.w3.org/ns/prov#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n# 现有三元组已包含以下信息，无需重复生成\nm:广州鸿蒙 m:主营业务 "税务局客户信息化服务" ;\n    m:企业年龄 "15" ;\n    m:成立年份 "2008" .'

In [6]:
await kg.extract("《赘婿》是一本古代穿越网络小说，主角是现代人穿越过去，包括权谋、战争、武林，主角有宁毅和他几个老婆", user_id=user_id)

INFO:httpx:HTTP Request: POST https://api.pumpkinaigc.online/v1/embeddings "HTTP/1.1 200 OK"
INFO:OpenAIEmbeddings:嵌入 `《赘婿》是一本古代穿越网络小说，主角是现` 等 1 个文本
INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
INFO:illufly.memory.kg:[12345678:62ebbafe382e86c03f1f97e5c31fd34b] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:赘婿 m:主角关系 "宁毅和他的几个老婆" .
INFO:illufly.memory.kg:[12345678:891afb94f347f103faacf537e7594a25] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:赘婿 m:包含元素 "战争" .
INFO:illufly.memory.kg:[12345678:014ea7d08663ee4e3ce6b111ac1c9afd] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:赘婿 m:类型 "古代穿越网络小说" .
INFO:illufly.memory.kg:[12345678:3ce09a69bff906bfc96c97a6bcfce444] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:赘婿 m:主角 "宁毅" .
INFO:illufly.memory.kg:[12345678:9dde86baa899d964a326b380ac81c332] 加载到向量数据库 turtle_

'@prefix m: <http://illufly.com/12345678/memory#> .\n@prefix prov: <http://www.w3.org/ns/prov#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\nm:赘婿 m:类型 "古代穿越网络小说" ;\n    m:主角 "宁毅" ;\n    m:包含元素 "权谋", "战争", "武林" ;\n    m:主角关系 "宁毅和他的几个老婆" .'

In [7]:
list(kg.graph)

[(rdflib.term.URIRef('http://illufly.com/12345678/memory#赘婿'),
  rdflib.term.URIRef('http://illufly.com/12345678/memory#主角关系'),
  rdflib.term.Literal('宁毅和他的几个老婆')),
 (rdflib.term.URIRef('http://illufly.com/12345678/memory#广州鸿蒙'),
  rdflib.term.URIRef('http://illufly.com/12345678/memory#主营业务'),
  rdflib.term.Literal('税务局客户信息化服务')),
 (rdflib.term.URIRef('http://illufly.com/12345678/memory#赘婿'),
  rdflib.term.URIRef('http://illufly.com/12345678/memory#包含元素'),
  rdflib.term.Literal('战争')),
 (rdflib.term.URIRef('http://illufly.com/12345678/memory#赘婿'),
  rdflib.term.URIRef('http://illufly.com/12345678/memory#包含元素'),
  rdflib.term.Literal('武林')),
 (rdflib.term.URIRef('http://illufly.com/12345678/memory#赘婿'),
  rdflib.term.URIRef('http://illufly.com/12345678/memory#类型'),
  rdflib.term.Literal('古代穿越网络小说')),
 (rdflib.term.URIRef('http://illufly.com/12345678/memory#赘婿'),
  rdflib.term.URIRef('http://illufly.com/12345678/memory#主角'),
  rdflib.term.Literal('宁毅')),
 (rdflib.term.URIRef('http://illu

In [8]:
print(await kg.query("软件", user_id=user_id, limit=4))

INFO:httpx:HTTP Request: POST https://api.pumpkinaigc.online/v1/embeddings "HTTP/1.1 200 OK"
INFO:OpenAIEmbeddings:嵌入 `软件` 等 1 个文本


(广州鸿蒙 主营业务 税务局客户信息化服务)
(赘婿 类型 古代穿越网络小说)


In [9]:
print(await kg.query("税务", user_id))

INFO:httpx:HTTP Request: POST https://api.pumpkinaigc.online/v1/embeddings "HTTP/1.1 200 OK"
INFO:OpenAIEmbeddings:嵌入 `税务` 等 1 个文本


(广州鸿蒙 主营业务 税务局客户信息化服务)
(广州鸿蒙 企业年龄 15)


In [10]:
await kg.extract("广州鸿蒙成立于2007年，是一家18年的信息化企业，专门从事税务信息化等软件开发业务。广州鸿蒙的董事长是王春辉。", user_id=user_id)

INFO:httpx:HTTP Request: POST https://api.pumpkinaigc.online/v1/embeddings "HTTP/1.1 200 OK"
INFO:OpenAIEmbeddings:嵌入 `广州鸿蒙成立于2007年，是一家18年的` 等 1 个文本
INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
INFO:illufly.memory.kg:[12345678:f01946cd536947212875f65fc9a1baf0] 加载到向量数据库 turtle_text: 
@prefix prov: <http://www.w3.org/ns/prov#> .

[] a prov:Activity .
INFO:illufly.memory.kg:[12345678:4aed36116f2dc27b64e187ce73803116] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .

m:广州鸿蒙 m:董事长 "王春辉" .
INFO:illufly.memory.kg:[12345678:267903456a18a0a567d87d4c552dbb56] 加载到向量数据库 turtle_text: 
@prefix m: <http://illufly.com/12345678/memory#> .
@prefix prov: <http://www.w3.org/ns/prov#> .

m:广州鸿蒙 prov:wasInvalidatedBy [ ] .
INFO:illufly.memory.kg:[12345678:f01946cd536947212875f65fc9a1baf0] 加载到向量数据库 turtle_text: 
@prefix prov: <http://www.w3.org/ns/prov#> .

[] a prov:Activity .
INFO:illufly.memory.kg:[12345678:25c5

'@prefix m: <http://illufly.com/12345678/memory#> .\n@prefix prov: <http://www.w3.org/ns/prov#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n# 过期三元组\n_:activity1 a prov:Activity ;\n    prov:invalidatedPredicate m:成立年份 ;\n    prov:invalidatedObject "2008" ;\n    prov:atTime "2023-01-01T00:00:00Z"^^xsd:dateTime ;\n    rdfs:label "成立年份更正" .\nm:广州鸿蒙 m:成立年份 "2008" ;\n    prov:wasInvalidatedBy _:activity1 .\n\n_:activity2 a prov:Activity ;\n    prov:invalidatedPredicate m:企业年龄 ;\n    prov:invalidatedObject "15" ;\n    prov:atTime "2023-01-01T00:00:00Z"^^xsd:dateTime ;\n    rdfs:label "企业年龄更新" .\nm:广州鸿蒙 m:企业年龄 "15" ;\n    prov:wasInvalidatedBy _:activity2 .\n\n# 新三元组\nm:广州鸿蒙 m:成立年份 "2007" ;\n    m:企业年龄 "18" ;\n    m:主营业务 "税务信息化等软件开发业务" ;\n    m:董事长 "王春辉" .'

In [11]:
print(await kg.query("鸿蒙", user_id, limit=120))

INFO:httpx:HTTP Request: POST https://api.pumpkinaigc.online/v1/embeddings "HTTP/1.1 200 OK"
INFO:OpenAIEmbeddings:嵌入 `鸿蒙` 等 1 个文本


(赘婿 包含元素 武林)
(广州鸿蒙 董事长 王春辉)
(ncc9a0cfeaa7d4bfcab4c553a9b890232b1 atTime 2023-01-01T00:00:00+00:00)
(广州鸿蒙 成立年份 2007)
(nddb9861160114435b334602ae62557b7b1 label 成立年份更正)
(赘婿 类型 古代穿越网络小说)
(n5b1f0e75461b41eb89acc98395a77bf1b1 invalidatedObject 2008)
(n4c0ea785e3d44f8ba3d04f47d963a484b1 type Activity)
(n3002fd514a5e40b495e75e91b1c74712b1 type Activity)
(赘婿 包含元素 权谋)
(广州鸿蒙 wasInvalidatedBy n5050179398924be29832e281e5d14727b1)
(广州鸿蒙 成立年份 2008)
(广州鸿蒙 主营业务 税务信息化等软件开发业务)
(广州鸿蒙 wasInvalidatedBy n611a787bb1f14511a8594eda69a89642b1)
(nbf673bf2b6204cb6b0cf09cb5f9c0fa0b1 label 企业年龄更新)
(赘婿 主角关系 宁毅和他的几个老婆)
(赘婿 包含元素 战争)
(广州鸿蒙 主营业务 税务局客户信息化服务)
(n1dfae183d79049aabe174e13119f11acb1 invalidatedPredicate 企业年龄)
(ne1e2354a441c4accae7a635be4c266eeb1 atTime 2023-01-01T00:00:00+00:00)
(广州鸿蒙 企业年龄 18)
(ne4fe20f00b124c71b635dd09a8287831b1 invalidatedPredicate 成立年份)
(n26727526f2f2417fa9defa64a7c48345b1 invalidatedObject 15)
(赘婿 主角 宁毅)
(广州鸿蒙 企业年龄 15)
